# 크롤링하기
## Driver 객체 만들기 
- DhromeDriverManager()에 의해 알맞는 크롬 브라우저 드라이버를 알아서 다운받는다. 
- driver 객체는 Selenium 패키지 내 Webdriver 클래스 객체로써 크롬 브라우저 그 자체를 의미한다. 
- driver 객체에는 우리가 손으로 할 수 있는 작업(창 최대화, 최소화, 창 닫기 등)을 위한 메서드도 존재한다.
- driver 객체에는 브라우저 창을 띄우지 않고 백그라운드에서 수행될 수 있도록 옵션도 가능하다.
> options = webdriver.ChromeOptions() <br/>
options.add_argument("headless") <br/>
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

In [89]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
url = 'https://m.map.naver.com/search2/search.naver'
driver.get(url)

## 검색어 입력하기
### 검색창 element 찾기
- 검색어를 입력하기 위해 검색 창이 어떤 element인지를 찾아야 한다. 
- 검색창 element를 찾아보자 

In [90]:

element = driver.find_element(by=By.CSS_SELECTOR , value='.Nbox_input_text._keyword')
print(element.get_attribute('title'))

검색어 입력


### 키워드 타이핑 및 엔터치기
- 브라우저를 직접 확인해보면 검색창을 한번 누르면 화면이 바뀌는 것을 볼 수 있고<br>
바뀐 화면에서 검색입력이 되는 것을 알 수 있다. 
- 이것을 코드로 그대로 재현해보자.


In [91]:
from selenium.webdriver.common.keys import Keys

search_keywords = '강릉맛집'
element.click()
element = driver.find_element(by=By.CSS_SELECTOR , value='.Nbox_input_text._search_input')
element.send_keys(search_keywords)         # 키워드 입력
element.send_keys(Keys.ENTER)         # 키워드 입력

# Keys 클래스에 어떤 값들이 있는지 확인해보자 


## 마켓 마스터 파일 만들기 
### 마켓 정보 크롤링하기 
- 마켓별 리뷰 데이터 크롤링 전 마켓에 대한 마스터 파일을 만들어 보도록 하자.
- 마켓 마스터 파일은 마켓 ID(data_id), 마켓 이름, 주소를 담을 예정이다.


In [60]:
element_lst = driver.find_elements(by=By.CSS_SELECTOR, value='li._item._lazyImgContainer')

market_name_lst = []
data_id_lst = [] 
address_lst = []
for element in element_lst:
    market_name_lst.append(element.get_attribute('data-title'))
    data_id_lst.append(element.get_attribute('data-id'))
    address_element = element.find_element(by=By.CSS_SELECTOR, value='.item_address._btnAddress')
    address_lst.append(address_element.text.replace('주소보기','').replace('\n',''))
    
print(market_name_lst[:3])
print(data_id_lst[:3])
print(address_lst[:3])
    



['강릉짬뽕순두부 동화가든 본점', '빅픽처6', '은파횟집']
['11859878', '37203388', '11587704']
['강원특별자치도 강릉시 초당순두부길77번길 15', '강원특별자치도 강릉시 주문진읍 해안로 1585', '강원특별자치도 강릉시 창해로350번길 29']


### 마켓 마스터 파일 저장하기 


In [61]:
# 디렉토리 존재 확인
import os

PROJ_HOME = '\\'.join(os.getcwd().split('\\')[:-1])
CRAWLED_DIR = PROJ_HOME + '\\crawled_files'
if not os.path.exists(CRAWLED_DIR):
    os.mkdir(CRAWLED_DIR)
    print(f'디렉토리 생성 완료({CRAWLED_DIR})')
else:
    print(f'디렉토리 존재({CRAWLED_DIR})')

디렉토리 존재(C:\Users\kalwr\PycharmProjects\NaverCrawling\crawled_files)


In [62]:
# 마켓 마스터 파일 저장
import pandas as pd 

market_mst_pd = pd.DataFrame({'data_id':data_id_lst, 'market_name':market_name_lst, 'address':address_lst})
print(market_mst_pd)
market_mst_pd.to_csv(CRAWLED_DIR + '\\market_master.csv', index=False, header=True, encoding='utf-8')
print(f'마켓 마스터 파일 저장 완료 ({market_mst_pd.shape[0]}건)')

       data_id      market_name                        address
0     11859878  강릉짬뽕순두부 동화가든 본점      강원특별자치도 강릉시 초당순두부길77번길 15
1     37203388             빅픽처6      강원특별자치도 강릉시 주문진읍 해안로 1585
2     11587704             은파횟집        강원특별자치도 강릉시 창해로350번길 29
3    998885728           카페 툇마루    강원특별자치도 강릉시 난설헌로 232 카페 툇마루
4   1167659600             초당버거  강원특별자치도 강릉시 초당순두부길77번길 20 지하층
..         ...              ...                            ...
70  1419683252           강릉역 옹막          강원특별자치도 강릉시 중기2길 9 1층
71    15380605              해미가            강원특별자치도 강릉시 솔올로 103
72  1025642664        팔도전복해물뚝배기  강원특별자치도 강릉시 창해로 375 강문동 234-2
73  1615572971              샌마르    강원특별자치도 강릉시 문화의길 8 1층 샌마르피자
74  1085613455             여포갈비        강원특별자치도 강릉시 하평길 64 여포갈비

[75 rows x 3 columns]
마켓 마스터 파일 저장 완료 (75건)


## 마켓별 리뷰 크롤링하기
- 특정 마켓으로 이동하면 url 이 https://m.place.naver.com/restaurant/{data_id}/home 로 변경되는 것을 볼 수 있다. 
- 따라서 마켓 마스터 파일에서 data_id를 읽은 후 url 주소로 이동 -> 크롤링 순서로 코드를 작성해보자 

### 함수 만들기
- data_id를 파라미터로 받아 리뷰 크롤링 후 csv로 저장하는 함수를 만들어보자.
- 리뷰가 많은 경우 더보기 버튼이 있다. 이 버튼을 눌러 리뷰 개수가 많이 나오도록 하자 
- 그런데 더보기 버튼을 끝까지 누르면 모든 리뷰데이터가 브라우저에 담기게 되어 성능 저하가 발생할 수 있다.
> out of Memory 발생 가능
- 따라서 리뷰 개수에 따라서 더보기 버튼을 조절하도록 하자

In [82]:
import time 

def get_review_n_save(data_id, market_name):
    if os.path.exists(f'{CRAWLED_DIR}\\{data_id}.csv'):
        print(f'{CRAWLED_DIR}\\{data_id}.csv 파일 존재, skip')
        return
    url = f'https://m.place.naver.com/restaurant/{data_id}/home'
    driver.get(url)
    a_tag_to_review_element = driver.find_element(by=By.CSS_SELECTOR, value='a.place_bluelink')
    a_tag_to_review_element.click()
    time.sleep(3)
    
    see_more_element = driver.find_element(by=By.CSS_SELECTOR, value='a.fvwqf')
    while True:
        li_review_element_lst = driver.find_elements(by=By.CSS_SELECTOR, value='li.YeINN')
        if len(li_review_element_lst) >= 1000:
            break
        else:
            time.sleep(1)
            try:
                see_more_element.click()
            except Exception as e:
                print(str(e))
                print(f'{market_name}: 리뷰 데이터 {len(li_review_element_lst)}개, 더보기 Loop break')
                break
    print(f'{market_name}: 리뷰 데이터 {len(li_review_element_lst)}개 준비 완료')
    
    # 리뷰 추출하여 저장하기 
    review_lst = [] 
    for li_review_element in li_review_element_lst:
        try:
            review_text = li_review_element.find_element(by=By.CSS_SELECTOR, value='span.zPfVt').text
            review_lst.append(review_text)
        except:
            # 텍스트를 남기지 않고 사진만 올린 경우 class=zPfVt Element가 없음 
            continue
    review_df = pd.DataFrame({'review_text':review_lst})
    review_df.to_csv(f'{CRAWLED_DIR}\\{data_id}.csv', index=False, header=True, encoding='utf-8')
    print(f'{market_name}: 리뷰 데이터({len(review_lst)}건) 저장 완료({CRAWLED_DIR}\\{data_id}.csv)')


### 마켓 마스터 파일에서 데이터프레임 불러오기

In [64]:
market_pd = pd.read_csv(CRAWLED_DIR + '\\market_master.csv')
print(market_pd)


       data_id      market_name                        address
0     11859878  강릉짬뽕순두부 동화가든 본점      강원특별자치도 강릉시 초당순두부길77번길 15
1     37203388             빅픽처6      강원특별자치도 강릉시 주문진읍 해안로 1585
2     11587704             은파횟집        강원특별자치도 강릉시 창해로350번길 29
3    998885728           카페 툇마루    강원특별자치도 강릉시 난설헌로 232 카페 툇마루
4   1167659600             초당버거  강원특별자치도 강릉시 초당순두부길77번길 20 지하층
..         ...              ...                            ...
70  1419683252           강릉역 옹막          강원특별자치도 강릉시 중기2길 9 1층
71    15380605              해미가            강원특별자치도 강릉시 솔올로 103
72  1025642664        팔도전복해물뚝배기  강원특별자치도 강릉시 창해로 375 강문동 234-2
73  1615572971              샌마르    강원특별자치도 강릉시 문화의길 8 1층 샌마르피자
74  1085613455             여포갈비        강원특별자치도 강릉시 하평길 64 여포갈비

[75 rows x 3 columns]


### 마켓 리뷰 데이터 수집 시작 
- 위에서 정의한 함수를 이용하여 크롤링 시작
- 시작하기 전 화면이 잠기지 않도록 화면 끄기 설정 off 필요

In [92]:
for idx, row in market_pd.iterrows():
    data_id = row['data_id']
    market_name = row['market_name']
    get_review_n_save(data_id, market_name)


C:\Users\kalwr\PycharmProjects\NaverCrawling\crawled_files\11859878.csv 파일 존재, skip
C:\Users\kalwr\PycharmProjects\NaverCrawling\crawled_files\37203388.csv 파일 존재, skip
Message: stale element reference: stale element not found
  (Session info: chrome=114.0.5735.199); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00F6A813+48355]
	(No symbol) [0x00EFC4B1]
	(No symbol) [0x00E05358]
	(No symbol) [0x00E087A1]
	(No symbol) [0x00E099E1]
	(No symbol) [0x00E09A80]
	(No symbol) [0x00E31D29]
	(No symbol) [0x00E29DAD]
	(No symbol) [0x00E4A73C]
	(No symbol) [0x00E29A36]
	(No symbol) [0x00E4AA94]
	(No symbol) [0x00E5C922]
	(No symbol) [0x00E4A536]
	(No symbol) [0x00E282DC]
	(No symbol) [0x00E293DD]
	GetHandleVerifier [0x011CAABD+2539405]
	GetHandleVerifier [0x0120A78F+2800735]
	GetHandleVerifier [0x0120456C+2775612]
	GetHandleVerifier [0x00FF51E0+6161

KeyboardInterrupt: 